# Session 4: 모델 아티팩트 관리 및 레지스트리 등록

### ■ Session Objective
- Workbench 로컬 환경에 생성된 **모델 아티팩트(Model Artifacts)**를 **영구 저장소(Persistent Storage)**인 GCS로 이전하고, 이를 Vertex AI Model Registry에 등록하여 중앙 관리형 배포 자산으로 변환합니다.

---

### ■ Session Workflow (Simplified)
![Session 4 Workflow](../images/session4-workflow.png)

### ■ Key Concepts

**1. 모델 아티팩트 (Model Artifacts)**
-   **정의:** 머신러닝 학습 프로세스의 최종 산출물로, 모델을 재현하거나 배포하는 데 필요한 **모든 파일의 집합**을 의미합니다.
-   **본 실습의 아티팩트:**
    -   `loan_approval_model.pkl`: 직렬화된(Serialized) 모델 객체 자체.
    -   `model_features.pkl`: 추론 시점의 입력 데이터 형식을 보장하기 위한 피처(컬럼) 정보.

**2. 영구 저장소 (Persistent Storage) vs 임시 저장소 (Ephemeral Storage)**
-   **임시 저장소 (Workbench 로컬 디스크):**
    -   Workbench 인스턴스(VM)에 종속된 디스크입니다.
    -   인스턴스 중지/삭제 시 데이터에 접근할 수 없거나 유실될 위험이 있습니다. **(휘발성)**
-   **영구 저장소 (Google Cloud Storage - GCS):**
    -   컴퓨팅 인스턴스와 완전히 분리된 고내구성 오브젝트 스토리지입니다. **(비휘발성)**
    -   모델 아티팩트, 원본 데이터 등 핵심 자산을 영구적으로 보관하고 다른 서비스와 공유하기 위한 **'중앙 자산 창고'** 역할을 수행합니다.

**3. Vertex AI Model Registry의 역할**
-   Model Registry는 GCS에 저장된 아티팩트를 **배포 가능한 서비스 단위**로 추상화하는 **카탈로그**입니다.
-   모델 파일의 **GCS 경로(위치)**와 해당 모델을 실행할 **환경(컨테이너)**을 매핑하여 버전을 관리합니다.

---

In [10]:
# ==============================================================================
# [실습 1] GCS에 모델 아티팩트 업로드 (Upload Artifacts to GCS)
# ==============================================================================
# 
# '!' 문자는 Jupyter Notebook에서 터미널(Shell) 명령어를 직접 실행하라는 의미입니다.
# 'gsutil'은 Google Cloud Storage를 다루는 강력한 커맨드라인 툴입니다.
# 
# 'cp' (copy) 명령어에 '-r' (recursive, 재귀적) 옵션을 사용하여,
# Workbench 로컬의 'models/' 폴더와 그 안의 모든 파일들을
# GCS 버킷 내의 'models/' 경로로 통째로 복사합니다.
# 
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※
# ※ [YOUR-GCS-BUCKET-NAME] 부분은 
# ※ Session 3에서 사용했던 본인의 GCS 버킷 이름으로 정확히 변경해야 합니다.
# ※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※※

# 1. 수강생이 직접 수정해야 하는 변수
BUCKET_NAME = "my_instance-ysh2-banking-data" # 예: "여러분의-GCS-버킷-이름"

# # 2. gsutil rsync 명령어 실행
# print(f"'{BUCKET_NAME}' 버킷의 'models/' 폴더와 로컬 'models/' 폴더를 동기화(rsync)합니다...")
# print("GCS에만 존재하는 불필요한 파일/폴더는 자동으로 삭제됩니다.")

# !gsutil rsync -r -d models/ gs://{BUCKET_NAME}/models/

# print("\n==============================================================================")
# print(f"▶ 동기화(rsync) 완료!")
# print(f"▶ 지금 바로 GCP 콘솔의 Cloud Storage 브라우저로 이동하여")
# print(f"▶ '{BUCKET_NAME}' 버킷 안에 'models/' 폴더가 '하나만' 있고,")
# print(f"▶ 그 안에 .pkl 파일 2개만 깔끔하게 들어있는지 확인하세요.")
# print("==============================================================================")

# 2. 업로드할 경로 정의
#    Model Registry에 등록할 모델 파일 전용 경로 (버전 1)
MODEL_GCS_PATH = f"gs://{BUCKET_NAME}/loan_model/v1/"
#    나중에 사용할 피처 목록 파일 경로
ARTIFACT_GCS_PATH = f"gs://{BUCKET_NAME}/loan_model/artifacts/"

# 3. GCS로 각 파일을 분리하여 업로드
print(f"모델 파일 업로드 중... -> {MODEL_GCS_PATH}")
!gsutil cp models/model.pkl {MODEL_GCS_PATH}

print(f"\n피처 목록 파일 업로드 중... -> {ARTIFACT_GCS_PATH}")
!gsutil cp models/model_features.pkl {ARTIFACT_GCS_PATH}

print("\n==============================================================================")
print(f"▶ 업로드 완료!")
print(f"▶ GCS 버킷에서 'loan_model/v1/' 폴더와 'loan_model/artifacts/' 폴더가")
print(f"▶ 각각 생성되고 파일이 1개씩 들어갔는지 확인하세요.")
print("==============================================================================")

모델 파일 업로드 중... -> gs://my_instance-ysh2-banking-data/loan_model/v1/
Copying file://models/model.pkl [Content-Type=application/octet-stream]...
- [1 files][  1.9 MiB/  1.9 MiB]                                                
Operation completed over 1 objects/1.9 MiB.                                      

피처 목록 파일 업로드 중... -> gs://my_instance-ysh2-banking-data/loan_model/artifacts/
Copying file://models/model_features.pkl [Content-Type=application/octet-stream]...
/ [1 files][  270.0 B/  270.0 B]                                                
Operation completed over 1 objects/270.0 B.                                      

▶ 업로드 완료!
▶ GCS 버킷에서 'loan_model/v1/' 폴더와 'loan_model/artifacts/' 폴더가
▶ 각각 생성되고 파일이 1개씩 들어갔는지 확인하세요.


# [실습 2] Vertex AI Model Registry 등록 (GCP Console UI)

이제 모델 파일이 영구 저장소(GCS)에 안전하게 보관되었으므로, Vertex AI가 이 모델을 '자산'으로 인식하고 관리할 수 있도록 Model Registry에 등록합니다.

이 작업은 GCP 콘솔의 웹 인터페이스(UI)를 통해 진행합니다.

---

### ■ 등록 절차 (Step-by-Step)

1.  **Vertex AI 대시보드 이동:**
    -   GCP 콘솔 상단 검색창에서 **'Vertex AI'**를 검색하여 대시보드로 이동합니다.

2.  **Model Registry 메뉴 선택:**
    -   왼쪽 탐색 메뉴에서 **모델 (Models)**을 클릭합니다.
   
![Session 4 Model Registry](../images/session4-modelregistry.png)

3.  **모델 등록 (Import) 시작:**
    -   페이지 상단의 **`+ 등록`** (또는 `+ IMPORT`) 버튼을 클릭합니다.

4.  **모델 정보 입력 (Form)**:

    -   **1. 기본 정보 (Define model):**
        -   **모델 이름:** `loan-approval-model` (모든 수강생이 이 이름을 사용하는 것을 권장합니다)
        -   **리전:** `asia-northeast3 (Seoul)`
        -   *나머지는 기본값으로 두고 '계속'을 클릭합니다.*

    -   **2. 모델 설정 (Model settings):**
        -   "모델 아티팩트 가져오기" 섹션에서 **"사전 빌드된 컨테이너"**가 선택되어 있는지 확인합니다.
        -   **모델 아티팩트 경로:** `찾아보기` 버튼을 클릭합니다.
            -   [실습 1]에서 업로드한 GCS 버킷(`여러분의-GCS-버킷-이름`)을 선택합니다.
            -   버킷 안의 **`models` 폴더를 클릭**한 뒤, 하단의 **`선택`** 버튼을 누릅니다.
            -   (경로가 `gs://여러분의-GCS-버킷-이름/loan_model/v1` 와 같이 **폴더 경로**로 지정되어야 합니다.)
        -   **예측 컨테이너:** `Scikit-learn`을 선택합니다.
        -   **프레임워크 버전:** `0.15` 을 선택합니다.

    -   **3. (옵션) 설명:**
        -   *이 단계는 건너뛰고 '가져오기' 버튼을 클릭합니다.*

5.  **등록 완료 확인:**
    -   '모델' 메뉴 목록에 방금 등록한 `loan-approval-model`이 표시됩니다.
    -   모델 이름을 클릭하여 상세 정보 페이지로 이동하면 **'기본 버전 1'**이 성공적으로 등록된 것을 확인할 수 있습니다.

---

### ■ Session 4 완료
축하합니다! 이제 여러분의 모델은 Vertex AI에 공식적으로 등록된 배포 가능 자산이 되었습니다.